In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!unzip /kaggle/input/denoising-dirty-documents/test.zip

In [ ]:
!unzip /kaggle/input/denoising-dirty-documents/train.zip

In [ ]:
!unzip /kaggle/input/denoising-dirty-documents/train_cleaned.zip

In [ ]:
train_filenames = os.listdir('train')
train_cleaned_filenames = os.listdir('train_cleaned')
test_filenames = os.listdir('test')
print("Train files: {}".format(len(train_filenames)))
print("Test files: {}".format(len(test_filenames)))
print("Cleaned Train files: {}".format(len(train_cleaned_filenames)))

In [ ]:
import cv2
train_images = [cv2.imread(os.path.join('train', file), 0) for file in train_filenames if file.endswith('.png')]
cleaned_train_images = [cv2.imread(os.path.join('train_cleaned', file), 0) for file in train_cleaned_filenames if file.endswith('.png')]
test_images = [cv2.imread(os.path.join('test', file), 0) for file in test_filenames if file.endswith('.png')]

train_images = [cv2.resize(img, (540, 260)) for img in train_images]
cleaned_train_images = [cv2.resize(img, (540, 260)) for img in cleaned_train_images]
test_images = [cv2.resize(img, (540, 260)) for img in test_images]

print(len(train_images))
print(len(cleaned_train_images))
print(len(test_images))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 20))
plt.subplot(1, 2, 1)
plt.imshow(train_images[10], cmap='gray')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(cleaned_train_images[10], cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
index = np.random.randint(0, len(train_images))
plt.subplot(1, 2, 1)
plt.imshow(train_images[index], cmap='gray')
plt.subplot(1, 2, 2)
plt.imshow(cv2.threshold(train_images[index], 120, 255, cv2.THRESH_BINARY)[1], cmap='gray')
plt.show()

In [ ]:
train_images = np.stack(train_images, axis=0).astype('float32')/255.
cleaned_train_images = np.stack(cleaned_train_images, axis=0).astype('float32')/255.
test_images = np.stack(test_images, axis=0).astype('float32')/255.

In [ ]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, UpSampling2D
from tensorflow.keras.models import Model

def encoder(input_img):
  x = Conv2D(32, (3, 3), activation='relu', padding='same', name='encoder_conv1')(input_img)
  x = MaxPool2D((2, 2), padding='same', name='encoder_pool1')(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same', name='encoder_conv2')(x)
  x = MaxPool2D((2, 2), padding='same', name='encoder_pool2')(x)
  encoded = Conv2D(128, (3, 3), activation='relu', padding='same', name='encoder_conv3')(x)
  return encoded


def decoder(encoded):
  x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decoder_conv1')(encoded)
  x = UpSampling2D((2, 2), name='decoder_upsample1')(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decoder_conv2')(x)
  x = UpSampling2D((2, 2), name='decoder_upsample2')(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same', name='decoder_conv3')(x)
  decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same', name='decoder_conv4')(x)
  return decoded

In [ ]:
input_img = Input(shape=(260, 540, 1))

#create autoencoder model
autoencoder = Model(input_img, decoder(encoder(input_img)))
#complile model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
earlyStop = EarlyStopping(monitor='loss', patience=10, mode='min')
chkPoint = ModelCheckpoint(filepath='best_weights.h5', monitor='loss', mode='min', save_best_only=True)
train_history = autoencoder.fit(np.expand_dims(train_images, 3), np.expand_dims(cleaned_train_images, 3),
                epochs=500,
                batch_size=32,
                shuffle=True,
                callbacks = [chkPoint, chkPoint]
               )

In [ ]:
autoencoder.load_weights('best_weights.h5')

test_image = test_images[60]
test_image = test_image.reshape((1, test_image.shape[0], test_image.shape[1], 1))
prediected_img = autoencoder.predict(test_image)

plt.figure(figsize=(20, 20))
plt.subplot(1, 2, 1)
plt.imshow(test_image[0, :, :, 0], cmap='gray')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(prediected_img[0, :, :, 0], cmap='gray')
plt.axis('off')
plt.show()